First we fetch the data:

In [ ]:
# import pandas as pd
# programmers_train_data = pd.read_csv('C:\\fibberio\\sandbox\\programmers-train.csv', skipinitialspace=True)
# programmers_test_data = pd.read_csv('C:\\fibberio\\sandbox\\programmers-test.csv', skipinitialspace=True)
# programmers_train_data = programmers_train_data.drop(['first_name', 'last_name'], axis=1)
# programmers_test_data = programmers_test_data.drop(['first_name', 'last_name'], axis=1)
# del programmers_train_data[programmers_train_data.columns[0]]
# del programmers_test_data[programmers_test_data.columns[0]]

In [ ]:
# programmers_test_data

In [ ]:
# from sklearn.model_selection import train_test_split

# target_column_name = "score"

# data_train = programmers_train_data
# data_test = programmers_test_data

In [1]:
target_feature = "score"

Now create an MLClient:

In [2]:
from azure.ml import MLClient
from azure.identity import DefaultAzureCredential
ml_client = MLClient.from_config(credential=DefaultAzureCredential(exclude_shared_token_cache_credential=True),
                     logging_enable=True)

Found the config file in: C:\RAI-vNext-Preview\config.json


In [3]:
# data_train.to_parquet("programmers-train.parquet")
# data_test.to_parquet("programmers-test.parquet")

In [4]:
programmers_version_string = '11'

Upload the datasets:

In [5]:
from azure.ml.entities import Data
from azure.ml.constants import AssetTypes

input_train_data = "Programmers_Train_Data"
input_test_data = "Programmers_Test_Data"

train_data = Data(
    path=f"programmers-train.parquet",
    type=AssetTypes.URI_FILE,
    description="RAI programmers training data",
    name=input_train_data,
    version=programmers_version_string,
)
ml_client.data.create_or_update(train_data)

test_data = Data(
    path=f"programmers-test.parquet",
    type=AssetTypes.URI_FILE,
    description="RAI programmers test data",
    name=input_test_data,
    version=programmers_version_string,
)
ml_client.data.create_or_update(test_data)

Data({'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'Programmers_Test_Data', 'description': 'RAI programmers test data', 'tags': {}, 'properties': {}, 'id': '/subscriptions/fac34303-435d-4486-8c3f-7094d82a0b60/resourceGroups/RAIPM/providers/Microsoft.MachineLearningServices/workspaces/RAIPM2/data/Programmers_Test_Data/versions/11', 'base_path': './', 'creation_context': <azure.ml._restclient.v2022_02_01_preview.models._models_py3.SystemData object at 0x00000181EA469250>, 'serialize': <msrest.serialization.Serializer object at 0x00000181EA45E910>, 'version': '11', 'latest_version': None, 'path': 'azureml://subscriptions/fac34303-435d-4486-8c3f-7094d82a0b60/resourcegroups/RAIPM/workspaces/RAIPM2/datastores/workspaceblobstore/paths/LocalUpload/ea1a713eaba89785ad8ecfb93cedb7f2/programmers-test.parquet', 'referenced_uris': None})

# Creating the Model

To simplify the model creation process, we're going to use a pipeline.

Before we do anything else, we need to specify the version of the RAI components:

In [6]:
version_string = '11'

Now we can create the training script:

In [7]:
import os

os.mkdir('component_src')

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'component_src'

In [8]:
%%writefile component_src/training_script_reg.py

import argparse
import os
import shutil
import tempfile


from azureml.core import Run

import mlflow
import mlflow.sklearn

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--training_data", type=str, help="Path to training data")
    parser.add_argument("--target_column_name", type=str, help="Name of target column")
    parser.add_argument("--model_output", type=str, help="Path of output model")

    # parse args
    args = parser.parse_args()

    # return args
    return args

def create_regression_pipeline(X, y):
    pipe_cfg = {
        'num_cols': X.dtypes[X.dtypes == 'int64'].index.values.tolist(),
        'cat_cols': X.dtypes[X.dtypes == 'object'].index.values.tolist(),
    }
    num_pipe = Pipeline([
        ('num_imputer', SimpleImputer(strategy='median')),
        ('num_scaler', StandardScaler())
    ])
    cat_pipe = Pipeline([
        ('cat_imputer', SimpleImputer(strategy='constant', fill_value='?')),
        ('cat_encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
    ])
    feat_pipe = ColumnTransformer([
        ('num_pipe', num_pipe, pipe_cfg['num_cols']),
        ('cat_pipe', cat_pipe, pipe_cfg['cat_cols'])
    ])

    # Append classifier to preprocessing pipeline.
    # Now we have a full prediction pipeline.
    pipeline = Pipeline(steps=[('preprocessor', feat_pipe),
                               ('model', LinearRegression())])
    return pipeline.fit(X, y)

def main(args):
    current_experiment = Run.get_context().experiment
    tracking_uri = current_experiment.workspace.get_mlflow_tracking_uri()
    print("tracking_uri: {0}".format(tracking_uri))
    mlflow.set_tracking_uri(tracking_uri)
    mlflow.set_experiment(current_experiment.name)
    
    #train_file = None
    #for filename in os.listdir(args.training_data):
    #    if filename.endswith('.csv'):
    #        train_file = filename
    #        break
    #print(train_file)
    #print(os.path.join(args.training_data, train_file))
    # Read in data
    print("Reading data")
    all_data = pd.read_parquet(args.training_data)

    print("Extracting X_train, y_train")
    print("all_data cols: {0}".format(all_data.columns))
    y_train = all_data[args.target_column_name]
    X_train = all_data.drop(labels=args.target_column_name, axis="columns")
    print("X_train cols: {0}".format(X_train.columns))

    print("Training model")
    # The estimator can be changed to suit
    model = create_regression_pipeline(X_train, y_train)

    # Saving model with mlflow - leave this section unchanged
    with tempfile.TemporaryDirectory() as td:
        print("Saving model with MLFlow to temporary directory")
        tmp_output_dir = os.path.join(td, "my_model_dir")
        mlflow.sklearn.save_model(sk_model=model, path=tmp_output_dir)

        print("Copying MLFlow model to output path")
        for file_name in os.listdir(tmp_output_dir):
            print("  Copying: ", file_name)
            # As of Python 3.8, copytree will acquire dirs_exist_ok as
            # an option, removing the need for listdir
            shutil.copy2(src=os.path.join(tmp_output_dir, file_name), dst=os.path.join(args.model_output, file_name))


# run script
if __name__ == "__main__":
    # add space in logs
    print("*" * 60)
    print("\n\n")

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")

Overwriting component_src/training_script_reg.py


Now, we want to place this into a component:

In [9]:
from azure.ml.entities import load_component

yaml_contents = f"""
$schema: http://azureml/sdk-2-0/CommandComponent.json
name: rai_programmers_training_component
display_name: Programmers training component for RAI example
version: {programmers_version_string}
type: command
inputs:
  training_data:
    type: path
  target_column_name:
    type: string
outputs:
  model_output:
    type: path
code: ./component_src/
environment: azureml:AML-RAI-Environment:1
""" + r"""
command: >-
  python training_script_reg.py
  --training_data ${{{{inputs.training_data}}}}
  --target_column_name ${{{{inputs.target_column_name}}}}
  --model_output ${{{{outputs.model_output}}}}
"""

yaml_filename = "ProgrammersRegTrainingComp.yaml"

with open(yaml_filename, 'w') as f:
    f.write(yaml_contents.format(yaml_contents))
    
train_component_definition = load_component(
    yaml_file=yaml_filename
)

ml_client.components.create_or_update(train_component_definition)

CommandComponent({'auto_increment_version': False, 'is_anonymous': False, 'name': 'rai_programmers_training_component', 'description': None, 'tags': {}, 'properties': {}, 'id': '/subscriptions/fac34303-435d-4486-8c3f-7094d82a0b60/resourceGroups/RAIPM/providers/Microsoft.MachineLearningServices/workspaces/RAIPM2/components/rai_programmers_training_component/versions/11', 'base_path': './', 'creation_context': <azure.ml._restclient.v2022_02_01_preview.models._models_py3.SystemData object at 0x00000181EB5DA9D0>, 'serialize': <msrest.serialization.Serializer object at 0x00000181EB5DAD90>, 'command': 'python training_script_reg.py --training_data ${{inputs.training_data}} --target_column_name ${{inputs.target_column_name}} --model_output ${{outputs.model_output}}', 'code': '/subscriptions/fac34303-435d-4486-8c3f-7094d82a0b60/resourceGroups/RAIPM/providers/Microsoft.MachineLearningServices/workspaces/RAIPM2/codes/92dc7ea3-eb37-4588-b116-cbe50166ce64/versions/1', 'environment_variables': None

In [10]:
# from azure.ml.entities import Code, CommandComponent

# training_code = Code(
#     local_path='training_script_reg.py'
# )

# training_inputs = {
#     'training_data': { 'type': 'path'},
#     'target_column_name': { 'type': 'string'}
# }

# training_outputs = {
#     'model_output': { 'type': 'path'}
# }

# training_component = CommandComponent(
#     name="ProgrammersTestRegTrainingComponent",
#     version="5",
#     display_name="Simple reg training component",
#     code=training_code,
#     environment=f"AML-RAI-Environment:1",
#     inputs=training_inputs,
#     outputs=training_outputs,
#     command="python training_script_reg.py " \
#             "--training_data ${{inputs.training_data}} " \
#             "--target_column_name ${{inputs.target_column_name}} " \
#             "--model_output ${{outputs.model_output}}"
# )

# ml_client.components.create_or_update(training_component)

# Running a training pipeline
Now we have a script which can train a model, we need to run it:

In [11]:
compute_name = "cpucluster"

from azure.ml.entities import AmlCompute

all_compute_names = [x.name for x in ml_client.compute.list()]

if compute_name in all_compute_names:
    print(f"Found existing compute: {compute_name}")
else:
    my_compute = AmlCompute(
        name=compute_name,
        size="Standard_DS2_v2",
        min_instances=0,
        max_instances=4,
        idle_time_before_scale_down=3600
    )
    ml_client.compute.begin_create_or_update(my_compute)
    print("Initiated compute creation")

Found existing compute: cpucluster


In [12]:
import time

model_name_suffix = int(time.time())
model_name = 'my_trained_reg_nb_model'

In [14]:
from azure.ml import dsl, Input

register_component = load_component(
    client=ml_client, name="register_model", version=version_string
)
train_model_component = load_component(
    client=ml_client, name="rai_programmers_training_component", version=programmers_version_string
)
programmers_train_pq = Input(
    type="uri_file", path=f"{input_train_data}:{programmers_version_string}", mode="download"
)
programmers_test_pq = Input(
    type="uri_file", path=f"{input_test_data}:{programmers_version_string}", mode="download"
)

@dsl.pipeline(
    compute=compute_name,
    description="Register Model for RAI Programmers example",
    experiment_name=f"RAI_Programmers_Example_Model_Training_{model_name_suffix}",
)
def my_training_pipeline(target_column_name, training_data):
    trained_model = train_component_definition(
        target_column_name=target_column_name,
        training_data=programmers_train_pq
    )

    _ = register_component(
        model_input_path=trained_model.outputs.model_output,
        model_base_name=model_name,
        model_name_suffix=model_name_suffix,
    )

    return {}

model_registration_pipeline_job = my_training_pipeline(target_feature, programmers_train_pq)

In [15]:
from azure.ml.entities import PipelineJob

def submit_and_wait(ml_client, pipeline_job) -> PipelineJob:
    created_job = ml_client.jobs.create_or_update(pipeline_job)
    assert created_job is not None

    while created_job.status not in ['Completed', 'Failed', 'Canceled', 'NotResponding']:
        time.sleep(30)
        created_job = ml_client.jobs.get(created_job.name)
        print("Latest status : {0}".format(created_job.status))
    assert created_job.status == 'Completed'
    return created_job

# This is the actual submission
training_job = submit_and_wait(ml_client, model_registration_pipeline_job)

Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Completed


In [16]:
expected_model_id = f'{model_name}_{model_name_suffix}:1'

In [17]:
fetch_model_component = load_component(
    client=ml_client, name='fetch_registered_model', version=version_string
)

rai_constructor_component = load_component(
    client=ml_client, name="rai_insights_constructor", version=version_string
)

rai_explanation_component = load_component(
    client=ml_client, name="rai_insights_explanation", version=version_string
)

rai_causal_component = load_component(
    client=ml_client, name="rai_insights_causal", version=version_string
)

rai_counterfactual_component = load_component(
    client=ml_client, name="rai_insights_counterfactual", version=version_string
)

rai_erroranalysis_component = load_component(
    client=ml_client, name="rai_insights_erroranalysis", version=version_string
)

rai_gather_component = load_component(
    client=ml_client, name="rai_insights_gather", version=version_string
)

In [18]:
import json

@dsl.pipeline(
        compute=compute_name,
        description="Example RAI computation on programmers data",
        experiment_name=f"RAI_Programmers_Example_RAIInsights_Computation_{model_name_suffix}",
    )
def rai_classification_pipeline(
        target_column_name,
        train_data,
        test_data,
    ):
        # Fetch the model
        fetch_job = fetch_model_component(
            model_id=expected_model_id
        )
        
        # Initiate the RAIInsights
        create_rai_job = rai_constructor_component(
            title="RAI Dashboard Example",
            task_type="regression",
            model_info_path=fetch_job.outputs.model_info_output_path,
            train_dataset=train_data,
            test_dataset=test_data,
            target_column_name=target_column_name,
            categorical_column_names='["location", "style", "job title", "OS", "Employer", "IDE", "Programming language"]'
        )
        
        # Add an explanation
        explain_job = rai_explanation_component(
            comment="Explanation for the programmers dataset",
            rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
        )
        
        # Add causal analysis
        causal_job = rai_causal_component(
            rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
            treatment_features='["Number of github repos contributed to", "YOE"]',
        )
        
        # Add counterfactual analysis
        counterfactual_job = rai_counterfactual_component(
            rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
            total_cfs=10,  # Bug filed - should be total_CFs,
            desired_range='[5, 10]'
        )
        
        # Add error analysis
        erroranalysis_job = rai_erroranalysis_component(
            rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
            filter_features='["style", "Employer"]'
        )
        
        # Combine everything
        rai_gather_job = rai_gather_component(
            constructor=create_rai_job.outputs.rai_insights_dashboard,
            insight_1=explain_job.outputs.explanation,
            insight_2=causal_job.outputs.causal,
            insight_3=counterfactual_job.outputs.counterfactual,
            insight_4=erroranalysis_job.outputs.error_analysis,
        )

        rai_gather_job.outputs.dashboard.mode = "upload"
        rai_gather_job.outputs.ux_json.mode = "upload"

        return {
            "dashboard": rai_gather_job.outputs.dashboard,
            "ux_json": rai_gather_job.outputs.ux_json,
        }

In [19]:
# Pipeline to construct the RAI Insights
insights_pipeline_job = rai_classification_pipeline(
    target_column_name=target_feature,
    train_data=programmers_train_pq,
    test_data=programmers_test_pq,
)

In [20]:
insights_job = submit_and_wait(ml_client, insights_pipeline_job)

Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Running
Latest status : Completed


In [ ]:
# THE END #

This is going to be a two component pipeline. The first will be the one we created above, which will train our model. The second will register it in AzureML:

In [ ]:
# The overall inputs for the pipeline

pipeline_inputs = {
    'target_column_name': target_column_name,
    'my_training_data': JobInput(dataset=f"Programmers_Train_Data:" + programmers_version_string, mode="download"),
    'my_test_data': JobInput(dataset=f"Programmers_Test_Data:" + programmers_version_string, mode="download")
}

# Specify the training job
train_job_inputs = {
    'target_column_name': '${{inputs.target_column_name}}',
    'training_data': '${{inputs.my_training_data}}',
}
train_job_outputs = {
    'model_output': None
}
train_job = ComponentJob(
    component=f"ProgrammersTestRegTrainingComponent:5",
    inputs=train_job_inputs,
    outputs=train_job_outputs
)

# The model registration job
register_job_inputs = {
    'model_input_path': '${{jobs.train-model-job.outputs.model_output}}',
    'model_base_name': model_name,
    'model_name_suffix': model_name_suffix
}
register_job_outputs = {
    'model_info_output_path': None
}
register_job = ComponentJob(
    component=f"RegisterModel:{version_string}",
    inputs=register_job_inputs,
    outputs=register_job_outputs
)

With our jobs specified, assemble them into a pipeline:

In [ ]:
model_registration_pipeline_job = PipelineJob(
    experiment_name=f"Register_Reg_Model_From_Notebook_01",
    description="Create and register a model from a notebook",
    jobs={
        'train-model-job': train_job,
        'register-model-job': register_job,
    },
    inputs=pipeline_inputs,
    outputs=register_job_outputs,
    compute="rai-cluster"
)

And submit it:

In [ ]:
from azure.ml.entities import PipelineJob

def submit_and_wait(ml_client, pipeline_job) -> PipelineJob:
    created_job = ml_client.jobs.create_or_update(pipeline_job)
    assert created_job is not None

    while created_job.status not in ['Completed', 'Failed', 'Canceled', 'NotResponding']:
        time.sleep(30)
        created_job = ml_client.jobs.get(created_job.name)
        print("Latest status : {0}".format(created_job.status))
    assert created_job.status == 'Completed'
    return created_job

In [ ]:
# This is the actual submission

training_job = submit_and_wait(ml_client, model_registration_pipeline_job)

# Creating the RAI Insights
We have a registered model, and can now run a pipeline to create the RAI insights. First off, compute the name of the model we registered (this is not straightforward since the Register Model component is used in testing):

In [ ]:
expected_model_id = f'{model_name}_{model_name_suffix}:1'

Now, we create the RAI pipeline itself. There are three 'component stages' in this pipeline:

1. Fetch the model
1. Construct an empty RAI dashboard
1. Run the RAI tool components

The job to fetch the registered model is:

In [ ]:
# This won't be necessary once models are types within the pipeline graph

fetch_job_inputs = {
    'model_id': expected_model_id
}
fetch_job_outputs = {
    'model_info_output_path': None
}
fetch_job = ComponentJob(
    component=f"FetchRegisteredModel:{version_string}",
    inputs=fetch_job_inputs,
    outputs=fetch_job_outputs
)

With this registered model (and our datasets), we can create an empty RAI dashboard:

In [ ]:
# Top level RAI Insights component

# We will reuse the same pipeline_inputs object in the end
create_rai_inputs = {
    'title': 'Run built from a Notebook',
    'task_type': 'regression',
    'model_info_path': '${{jobs.fetch-model-job.outputs.model_info_output_path}}',
    'train_dataset': '${{inputs.my_training_data}}',
    'test_dataset': '${{inputs.my_test_data}}',
    'target_column_name': '${{inputs.target_column_name}}',
    'categorical_column_names': '["location", "style", "job title", "OS", "Employer", "IDE", "Programming language"]',
}
create_rai_outputs = {
    'rai_insights_dashboard': None # Could theoretically redirect the datastore here
}
create_rai_job = ComponentJob(
    component=f"RAIInsightsConstructor:{version_string}",
    inputs=create_rai_inputs,
    outputs=create_rai_outputs
)

Now, create an instance of each of our RAI tools:

In [ ]:
# Setup the explanation
explain_inputs = {
   'comment': 'Insert text here',
    'rai_insights_dashboard': '${{jobs.create-rai-job.outputs.rai_insights_dashboard}}'
}
explain_outputs = {
    'explanation': None
}
explain_job = ComponentJob(
    component=f"RAIInsightsExplanation:{version_string}",
    inputs=explain_inputs,
    outputs=explain_outputs
)

# Setup causal
causal_inputs = {
    'rai_insights_dashboard': '${{jobs.create-rai-job.outputs.rai_insights_dashboard}}',
    'treatment_features': '["Number of github repos contributed to", "YOE"]'
}
causal_outputs = {
    'causal': None
}
causal_job = ComponentJob(
    component=f"RAIInsightsCausal:{version_string}",
    inputs=causal_inputs,
    outputs=causal_outputs
)

# Setup counterfactual
counterfactual_inputs = {
    'rai_insights_dashboard': '${{jobs.create-rai-job.outputs.rai_insights_dashboard}}',
    'total_CFs': '10',
    'desired_range': '[5, 10]'
}
counterfactual_outputs = {
    'counterfactual': None
}
counterfactual_job = ComponentJob(
    component=f"RAIInsightsCounterfactual:{version_string}",
    inputs=counterfactual_inputs,
    outputs=counterfactual_outputs
)

# Setup error analysis
error_analysis_inputs = {
    'rai_insights_dashboard': '${{jobs.create-rai-job.outputs.rai_insights_dashboard}}',
    'filter_features': '["style", "Employer"]'
}
error_analysis_outputs = {
    'error_analysis': None
}
error_analysis_job = ComponentJob(
    component=f"RAIInsightsErrorAnalysis:{version_string}",
    inputs=error_analysis_inputs,
    outputs=error_analysis_outputs
)

Now the 'gather' component which assembles everything into an `RAIInsights` object, and computes the JSON for the UX:

In [ ]:
# Configure the gather component
gather_inputs = {
    'constructor': '${{jobs.create-rai-job.outputs.rai_insights_dashboard}}',
    'insight_1': '${{jobs.explain-job.outputs.explanation}}',
    'insight_2': '${{jobs.causal-job.outputs.causal}}',
    'insight_3': '${{jobs.counterfactual-job.outputs.counterfactual}}',
    'insight_4': '${{jobs.error-analysis-job.outputs.error_analysis}}'
}
gather_outputs = {
    'dashboard': None,
    'ux_json': None
}
gather_job = ComponentJob(
    component=f"RAIInsightsGather:{version_string}",
    inputs=gather_inputs,
    outputs=gather_outputs
)

Finally, the pipeline itself:

In [ ]:
# Pipeline to construct the RAI Insights
insights_pipeline_job = PipelineJob(
    experiment_name=f"Compute_Insights_from_Notebook_{version_string}",
    description="Python submitted Orange insights using fetched model",
    jobs={
        'fetch-model-job': fetch_job,
        'create-rai-job': create_rai_job,
        'causal-job': causal_job,
        'counterfactual-job': counterfactual_job,
        'error-analysis-job': error_analysis_job,
        'explain-job': explain_job,
        'gather-job': gather_job
    },
    inputs=pipeline_inputs,
    outputs=None,
    compute="rai-cluster"
)

And submit it:

In [ ]:
insights_job = submit_and_wait(ml_client, insights_pipeline_job)

# Download and display the insights
Now we can download the insights we have computed. To start, we need to obtain the Run id of the 'gather-job' which ran as part of the previous pipeline. We have a helper for this, but the name of the experiment is required:

In [ ]:
from azure_ml_rai import list_rai_insights

In [ ]:
run_list = list_rai_insights(ml_client, insights_pipeline_job.experiment_name)

print(insights_pipeline_job.experiment_name)
display(run_list)

We can use the mini SDK to download to a local directory:

In [ ]:
from azure_ml_rai import download_rai_insights

download_dir = 'my_downloaded_insight'

download_rai_insights(
    ml_client,
    rai_insight_id=run_list[0],
    path=download_dir,
)

And with everything downloaded, we can load the RAIInsights object and instantiate the dashboard:

In [ ]:
from responsibleai import RAIInsights
from raiwidgets import ResponsibleAIDashboard

rai_i = RAIInsights.load(download_dir)

ResponsibleAIDashboard(rai_i)

If for some reason we only need the JSON file holding the contents of `RAIInsights.get_data()`, we can download the other output port of the 'Gather' component:

In [ ]:
from azure_ml_rai import download_rai_insights_ux

download_ux_dir = 'my_ux_insight'

download_rai_insights_ux(
    ml_client,
    rai_insight_id=run_list[0],
    path=download_ux_dir,
)